# Analiza wariantów

In [ ]:
import os                               # moduł OS języka Python
user_name = os.environ.get('USER')      # pobieramy zmienną środowiskową USER
bucket = f"gs://edugen-lab-{user_name}2" # konstruujemy sciezke dostepowa do pliku
print(bucket)

In [ ]:
# sprawdzenie czy dane są dostępne
! gsutil ls -r $bucket


## Przygotowanie sesji Spark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.getOrCreate()

## Stworzenie tabeli z wariantami

In [ ]:
var_path=f"{bucket}/vcf/motherHC.vcf"
table_var = 'variants'

spark.sql(f'DROP TABLE IF EXISTS {table_var}')

spark.sql(f'CREATE TABLE IF NOT EXISTS {table_var} \
USING org.biodatageeks.sequila.datasources.VCF.VCFDataSource \
OPTIONS(path "{var_path}")')

## Weryfikacja danych

<div class="alert alert-block alert-warning">
<b>Zadanie 4_1:</b> 
Napisz polecenie, które pokaże strukturę (kolumny i ich typy) tabeli z wariantami
    </div>

In [ ]:
spark.sql(f'select count (*) from {table_var}').show()

Widzimy, że ALT jest rodzaju tablicowego, sprawdźmy, czy są warianty mutlialleliczne?

In [ ]:
spark.sql(f'select * from {table_var} where size(alt) > 1').show()

In [ ]:
spark.sql(f'select * from {table_var} where size(alt) > 1').count() # ile ich jest ? 

In [ ]:
spark.sql(f'select genotypes from {table_var}').show(truncate=False) # tabela w postaci nieznormalizowanej -> mamy listę wartości

In [ ]:
var_anno_path=f"gs://edugen-common-data2/anno/HG002_annotated.tsv"
table_var_anno= 'variants_anno'
spark.sql(f'DROP TABLE IF EXISTS {table_var_anno}')
spark.sql(f'CREATE TABLE IF NOT EXISTS {table_var_anno} \
USING  CSV \
OPTIONS(HEADER true, INFERSCHEMA true, DELIMITER "\t" ) \
LOCATION  "{var_anno_path}"')

In [ ]:
anno = spark.sql(f"select * from {table_var_anno}")

<div class="alert alert-block alert-warning">
<b>Zadanie 4_2:</b> 
Wyświetl schemat DF anno i podaj liczbę kolumn. Jakiego typu są nowe kolumny?
    </div>

<div class="alert alert-block alert-warning">
<b>Zadanie 4_3:</b> 
Pogrupuj warianty po kolumnie IMPACT oraz oblicz ile jest wariantów w każdej grupie. Zrób to samo dla kolumny Consequence oraz SYMBOL.
    </div>


## Biblioteka Pandas

https://pandas.pydata.org/

Moduł Pandas jest biblioteką Pythonową do manipulacji danymi. W szczegolnosci w pandas mozemy stworzyc ramki danych i wykonywac na niej analize, agregacje oraz wizualizacje danych. 
Przy nieduzych zbiorach danych i prostych operacjach to doskonała biblioteka. Jednak kiedy zbior danych sie rozrasta lub kiedy wymagane sa zlozone transformacje to operacje moga byc wolne.

Operacje na rozproszonych danych sa szybsze. Ale tu takze napotykamy ograniczenia np trudność w wizualizacji danych.

In [ ]:
spark.sql(f"select * from {table_var_anno} LIMIT 10").toPandas()

**Ważne** Metoda toPandas() na ramce pyspark, konwertuje ramkę pyspark do ramki pandas. Wykonuje akcje pobrania wszystkich danych z executorów (z JVM) i transfer do  programu sterujacego (driver) i konwersje do typu Pythonowego w notatniku. Ze względu na ograniczenia pamięciowe w programie sterującym należy to wykonywać na podzbiorach danych.

Nie pokazują się wszystkie kolumny (w środku mamy ...). Aby wyświetlić całość musimy ustawić:

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
anno_pandas = anno.limit(10).toPandas()
anno_pandas

# Matplotlib
https://pandas.pydata.org/pandas-docs/version/0.23.4/api.html#api-dataframe-plotting

In [ ]:
anno2 = anno.groupby("ref").count().limit(10).toPandas()
anno2.plot.pie(y='count', labels=anno2['ref'],  autopct='%.0f')
plt.legend(loc="lower center")
plt.show()

<div class="alert alert-block alert-warning">
<b>Zadanie 4_4:</b> 
Przerób powyższe dwa rysunki, tak żeby pokazywały rozkład referencji tylko dla SNV (bez indeli).
    </div>



<div class="alert alert-block alert-warning">
<b>Zadanie 4_5:</b> 
Przygotuj pie chart dla kolumny Consequences. Uwzględnij jedną konsekwencję dla każdego wariantu.
    </div>



In [ ]:
spark.stop()